In [2]:
# импортируем NYTAPI из специализированной библиотеки по работе с API "Нью-Йорк Таймс"
from pynytimes import NYTAPI

# испортируем другие необходимые библиотеки
import time
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import datetime as datetime
from datetime import datetime as datetime_2

In [1]:
# определяем ключевые слова для поиска статей 
keywords_list = ['Jew', 'Jews', 'Jewish']

In [3]:
# Указываем свой API key (доступен после регистрации здесь: https://developer.nytimes.com/get-started)
api_key = "<используем свой API key>"

In [4]:
# Инициализируем NYT API
nyt = NYTAPI(api_key, parse_dates=True)

# Устанавливем даты начала и окончания временного периода для сканирования электронного архива "Нью-Йорк Таймс" (begin and end), задавая интервал для 
# одного скана цикоа программы. !!! Есть ограничение на количесвто запросов к электронному архиву "Нью-Йорк Таймс".
# Поэтому нельзя поставить сразу на весь период 1939-1945

begin = datetime_2(1939, 9, 1) 
end = datetime_2(1940, 8, 31)
# кладем их в словарь для использования как параметра запроса
date_dict = {"begin": begin, "end": end}

In [5]:
# словарь опций поиска
# содержит также указание тем/тегов статей, которыми ограничиваем выборку (поле "subject")

options_dict = {
    # Sort from earliest to latest
    "subject": ["MINORITIES AND OPPRESSED GROUPS", "MINORITIES, REFUGEES AND OPPRESSED GROUPS", "OCCUPATION AND REHABILITATION OF CONQUERED AREAS", "OCCUPIED COUNTRIES", "ATROCITIES"],
    "sort": "oldest",
    # Return only articles from New York Times (filters out other sources such as AP and Reuters)
    #"sources": ["New York Times"],
    # Return only news, analyses, and articles
    #"type_of_material": ["News Analysis", "News", "Article"]
}

In [ ]:
# основной цикл, в котором добавляем найденные метаданные искомых статей в список articles

articles = []
for i in range(7):
    for key_word in keywords_list:
        articles_for_cycle = nyt.article_search(
            query = key_word,
            results=250,
            dates=date_dict,
            options=options_dict)
        for item in articles_for_cycle:
            if item not in articles:
                articles.append(item)
        # Print len of aricles after each key word
        print(key_word + ": Length of list with aticles is " + str(len(articles)))
    begin += datetime.timedelta(days=366)
    end += datetime.timedelta(days=366)
    # Update a dictionary containing the datetime objects
    date_dict = {"begin": begin,  "end": end}
    # Print number of articles after each date interval
    print(str(i+1) + " step. " + "Rows in the list of articles: " + str(len(articles)) + " rows")
    time.sleep(5)

In [ ]:
# приводим JSON к DataFrame
df = pd.json_normalize(articles)

In [ ]:
# оставляем только необходимые поля (исключенные поля не заполнены почти)
df = df.loc[:,['abstract','web_url', 'print_page', 'keywords', 'pub_date', '_id', 'headline.main', 'byline.original', 'byline.person']]

In [ ]:
# сохраняем в формате csv
df.to_csv('Holocaust_in_NYT(9 columns).csv', sep='\t', encoding='utf-8', mode='a', header=True)